<center>
<h1 style="font-family:verdana">
 💻 🧑 Reconeixement d'entitats anomenades 🧑 💻


<p> 🎯 <b>Objectiu</b>: en aquesta segona part de la pràctica aprendrem a reconèixer entitats anomenades, és a dir, identificar i classificar entitats en una oració que poden ser (en el context de reserva de vols) la ciutat de sortida o d'arribada, la data del mes i el dia, si es de negocis o turista o altres categories específiques. Aquesta tasca se sol plantejar etiquetant cada paraula amb una etiqueta de la categoria de l'entitat a la qual correspon.

Tant la classificació d'intencions (tasca estudiada a la primera part) com el reconeixement d'entitats anomenades (Name Entity Recognition, NER) que veurem en aquesta part són components crucials dels sistemes de processament del llenguatge natural (Natural Language Processing, NLP) i sovint s'utilitzen junts per crear aplicacions d'intel·ligència artificial (IA) conversacionals més sofisticades. Per exemple, en un xatbot, la classificació d'intencions ajuda a entendre la intenció principal de l'usuari, mentre que el reconeixement d'entitats anomenades ajuda a extreure entitats rellevants per proporcionar respostes més contextualitzades.


<p> ✨ <b>Contingut</b>: els passos d'aquesta segona part seran similars a la primera. En primer lloc, analitzarem el conjunt de dades. En segon lloc, prepararem les dades per a poder entrenar el model. I finalment dissenyarem l'arquitectura i entrenarem el model. </p>


<p> ✏ <b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent. </p>

---

<h2> Índex </h2>

1. [Inspecció del conjunt de dades](#section-one)
  * [Exercici 1](#ex-one)
  * [Exercici 2](#ex-two)
2. [Preprocessament de dades](#section-two)
  * [Exercici 3](#ex-three)
  * [Exercici 4](#ex-four)
3. [Disseny del model i entrenament](#section-three)
  * [Exercici 5](#ex-five)
  * [Exercici 6](#ex-six)
4. [Lliurable](#section-four)

---

In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report


<h1><a name="section-one"> 1. Inspecció del conjunt de dades </a></h1>

In [9]:
!pip install gdown
!gdown "https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t"
!tar -zxf nlu_ATIS_data.tar.gz


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Downloading...
From: https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t
To: c:\Users\Usuario\Documents\Universitat\5_Quatri\TVD\frame-based\frame-based-system\NAME ENTITY RECOGNITION\nlu_ATIS_data.tar.gz

  0%|          | 0.00/122k [00:00<?, ?B/s]
100%|██████████| 122k/122k [00:00<00:00, 1.98MB/s]


In [10]:
!ls data

test.csv
train.csv


En aquesta segona part de la pràctica utilitzarem el mateix dataset que a la primera part, però en aquest cas utilitzarem la **primera columna** que correspon a les **oracions** introduïdes pels usuaris i la **segona columna** on trobarem les oracions d'entrada en format **BILOU**.

El format BILOU  és un esquema d'etiquetatge que es fa servir a les tasques de reconeixement d'entitats anomenades. El nom *BILOU* representa les etiquetes utilitzades en aquest esquema:

*   **B** - Beginning: primer token d'una entitat.
*   **I** - Inside: token dins d'una entitat.
*   **L** - Last: últim token de l'entitat.
*   **O** - Outside: token que no pertany a cap entitat.
*   **U** - Unit: entitats d'un sol token

A continuació, carregarem les dades per visualitzar i poder entendre millor aquestes etiquetes.

---

 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>

A continuació us demanem que carregueu els dos CSVs de la carpeta `data`: `train.csv`, `test.csv` utilitzant pandas. Recorda que aquests CSVs no tenen capçalera. Agafeu les 900 últimes lineas del fitxer `train.csv` per crear un dataframe per validació.

In [53]:
train_data = pd.read_csv('./data/train.csv', header=None)
val_data = train_data.tail(900)
train_data = pd.read_csv('./data/train.csv', header=None, nrows=4078)
test_data = pd.read_csv('./data/test.csv', header=None)


print('Training size:', len(train_data))
print('Validation dataset size:', len(val_data))
print('Test dataset size:', len(test_data))

Training size: 4078
Validation dataset size: 900
Test dataset size: 893


---

 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏</a></h1>

Tal com hem introduït abans, per a aquest exercici ens centrarem en la **primera** i la **segona** columna. Per tant, ara us demanem que guardeu en les següents variables, les llistes corresponents a les oracions i a les etiquetes de les tres particions (`train`, `validation` i `test`).

In [54]:
#TODO
import random
random_number = random.randint(0, len(train_data)-1)

train_sentences = list(train_data[0])
train_labels = list(s.replace('"', '') for s in train_data[1])
#train_labels = list(s.replace(' ', '') for s in train_labels)

print('Sentence: ', train_sentences[random_number])
print('Intent: ', train_labels[random_number])


val_sentences = list(val_data[0])
val_labels = list(s.replace('"', '') for s in val_data[1])
#val_labels = list(s.replace(' ', '') for s in val_labels)

test_sentences = list(test_data[0])
test_labels = list(s.replace('"', '') for s in test_data[1])
#test_labels = list(s.replace(' ', '') for s in test_labels)


Sentence:  what is the earliest flight from boston to san francisco on american airlines
Intent:   O O O B-flight_mod O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-airline_name I-airline_name


---

Si tot ha anat bé ja podem analitzar quin aspecte té el format BILOU.

In [55]:
print(train_sentences[5])
print(train_labels[5])

i need a flight tomorrow from columbus to minneapolis
 O O O O B-depart_date.today_relative O B-fromloc.city_name O B-toloc.city_name


Aquest hauria de ser el resultat obtingut si executeu la cel·la anterior:

```


i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
 "O O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O B-arrive_time.period_of_day"


```

En aquest exemple, `boston` i `838 am` corresponen a l'entitat "ciutat de sortida" i a l'entitat "hora de sortida" respectivament (`fromloc.city_name`,`depart_time.time`). `838` correspon al primer token que pertany la entitat `depart_time.time` i `am` al segon de la mateixa entitat. Els tokens com `i` o `want` no pertanyen a cap entitat.

In [56]:
print(train_sentences[1])
print(train_labels[1])

what flights are available from pittsburgh to baltimore on thursday morning
 O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day


I aquest hauria de ser el resultat de la cel·la anterior:

```
what flights are available from pittsburgh to baltimore on thursday morning
 O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day
```

En l'anterior exemple, `what`, `flights`, `are`, `available`, `from`, `to` i `on` estan etiquetades com a no pertanyents a cap entitat. I, en canvi, `pittsburgh` pertany a l'entitat d'un sol token anomenada ciutat d'arribada (`toloc.city_name`). També hi ha entitats compostes com `baltimore` que pertany a l'entitat ciutat de sortida (`fromloc.city_name`) i `thursday morning` que pertanyen a les entitats data de sortida (`depart_date.day_name`, `depart_time.period_of_day`).

Vegem-ne la llista completa d'entitats.

In [57]:
from collections import Counter

def count_unique_entities(list_of_label_sentences):
  flat_labels = []
  for labels in list_of_label_sentences:
    flat_labels += labels.split()
  c = Counter(flat_labels)
  return len(c), list(c.keys())

num_unique_entities, unique_entities = count_unique_entities(train_labels)

print("Number of different entities:", num_unique_entities)

print("Unique entities:", unique_entities)

Number of different entities: 119
Unique entities: ['O', 'B-fromloc.city_name', 'B-depart_time.time', 'I-depart_time.time', 'B-toloc.city_name', 'B-arrive_time.time', 'B-arrive_time.period_of_day', 'B-depart_date.day_name', 'B-depart_time.period_of_day', 'B-flight_time', 'I-flight_time', 'I-fromloc.city_name', 'B-cost_relative', 'B-round_trip', 'I-round_trip', 'B-fare_amount', 'I-fare_amount', 'B-depart_date.today_relative', 'I-toloc.city_name', 'B-city_name', 'B-stoploc.city_name', 'B-toloc.airport_code', 'B-depart_time.time_relative', 'B-class_type', 'I-class_type', 'B-depart_date.date_relative', 'B-airline_name', 'I-airline_name', 'B-arrive_time.time_relative', 'B-depart_time.start_time', 'I-depart_time.start_time', 'B-depart_time.end_time', 'I-depart_time.end_time', 'B-fromloc.airport_name', 'I-fromloc.airport_name', 'B-toloc.state_name', 'B-depart_date.day_number', 'I-depart_date.day_number', 'B-depart_date.month_name', 'B-mod', 'B-fare_basis_code', 'B-transport_type', 'B-flight_m

<h1><a name="section-two"> 2. Preprocessament de dades </a></h1>

El processament de les dades serà semblant al de la primera part d'aquesta pràctica.

---

 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏</a></h1>

En aquest exercici us demanem que realitzeu els passos següents per preparar les dades.

 1. El primer pas serà construir el vocabulari a partir de les paraules presents a les oracions d'entrenament.

2. El segon pas serà convertir les oracions en seqüències de nombres enters usant el tokenitzador.

3. El tercer pas serà guardar la longitud original de cada oració. Això ens serà útil per evaluar el nostre model sense tenir en compte el padding.

3. Finalment, per aconseguir que totes les seqüències tinguen la mateixa longitud, fixarem la longitud segons la màxima trobada a l'entrenament i afegirem zeros a les oracions de menor longitud.

Recordeu que podeu consultar com fer-ho a la documentació de la llibrería:
* <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer"> Tokenizer </a>
* <a href="https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences"> Pad Sequences </a>




In [58]:
#TODO
num_words=1000
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(train_sentences)

vocab = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
max_sequence_length = max(map(len, train_sequences))
train_pad_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding="post")

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_pad_sequences_old = pad_sequences(val_sequences, maxlen=max_sequence_length, padding="post")

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_pad_sequences_old = pad_sequences(test_sequences, maxlen=max_sequence_length, padding="post")

In [17]:
len(train_pad_sequences[0])

46

In [18]:
print(train_sequences[0])

[12, 69, 1, 38, 2, 9, 64, 415, 84, 17, 75, 16, 13, 64, 493, 16, 4, 36]



---



Tal com vam fer a la primera part, aquí també hem de convertir les diferents classes d'entitats en vectors one-hot.

---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏</a></h1>

Per aconseguir-ho haureu de seguir els passos següents.

 1. En primer lloc, haureu d'esbrinar quantes etiquetes diferents hi ha. Podeu prendre com a exemple la funció `count_unique_entities` per fer-ho. Tingueu en compte que haureu de modificar la funció, ja que per exemple aquesta funció considera que `B-depart_time.time`, `I-depart_time.time`, `L-depart_time.time`, `U-depart_time.time` són la mateixa entitat. En aquest exercici, necessitarem comptar-les per separat. També l'entitat O ha de ser considerada com una classe.

 2. El segon pas serà codificar les diferents classes trobades en etiquetes numèriques. Tingueu en compte que cada paraula de l'oració té una etiqueta i, per tant, per a cada oració tindrem una llista d'etiquetes. El *padding* el codificarem amb l'etiqueta corresponent a **O** (outside). Per ajudar-vos hem preparat el processament per a les etiquetes d'entrenament, intenteu comprendre el que es fa i així repetir-ho per a la partició de validació i test.

 3. Finalment, haureu de convertir les diferents classes a vectors one-hot. Recordeu de nou que per a cada oració tindrem una llista de vectors one-hot.


 Podeu consultar els apartats de la documentació:
 * <a href=https://www.tensorflow.org/guide/keras/understanding_masking_and_padding> Masking and Padding </a>

 * <a href=https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical> To Categorical </a>



In [19]:
#TODO
#Step 1
def count_unique_entities(list_of_label_sentences): # No jentenc lo de que haurem de modificar la funció
  flat_labels = []
  for labels in list_of_label_sentences:
    flat_labels += labels.split()
  c = Counter(flat_labels)
  return len(c), list(c.keys())


num_unique_entities, unique_entities = count_unique_entities(train_labels)

print("Number of different entities:", num_unique_entities)
print("Unique entities:", unique_entities)
print("U-depart_time.time" in unique_entities)


Number of different entities: 119
Unique entities: ['O', 'B-fromloc.city_name', 'B-depart_time.time', 'I-depart_time.time', 'B-toloc.city_name', 'B-arrive_time.time', 'B-arrive_time.period_of_day', 'B-depart_date.day_name', 'B-depart_time.period_of_day', 'B-flight_time', 'I-flight_time', 'I-fromloc.city_name', 'B-cost_relative', 'B-round_trip', 'I-round_trip', 'B-fare_amount', 'I-fare_amount', 'B-depart_date.today_relative', 'I-toloc.city_name', 'B-city_name', 'B-stoploc.city_name', 'B-toloc.airport_code', 'B-depart_time.time_relative', 'B-class_type', 'I-class_type', 'B-depart_date.date_relative', 'B-airline_name', 'I-airline_name', 'B-arrive_time.time_relative', 'B-depart_time.start_time', 'I-depart_time.start_time', 'B-depart_time.end_time', 'I-depart_time.end_time', 'B-fromloc.airport_name', 'I-fromloc.airport_name', 'B-toloc.state_name', 'B-depart_date.day_number', 'I-depart_date.day_number', 'B-depart_date.month_name', 'B-mod', 'B-fare_basis_code', 'B-transport_type', 'B-flight_m

In [20]:

#Step 2

label_encoder = LabelEncoder()


label = label_encoder.fit(["<pad>"]+list(unique_entities))
train_numerical_labels =[label_encoder.transform(t.split()) for t in train_labels]
train_pad_labels = pad_sequences(train_numerical_labels, maxlen=max_sequence_length, padding="post")


In [21]:
train_pad_labels[0]

array([119, 119, 119, 119, 119,  46, 119,  34,  95, 119, 119, 119,  73,
       119,  15, 119, 119,  13,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [22]:


def remove_sentences(list_labels, list_sequences):
  idx_to_remove = []
  labels_to_remove = []
  for idx, labels in enumerate(list_labels):
    for label in labels:
      if label not in unique_entities:
        print(label)
        idx_to_remove.append(idx)
        labels_to_remove.append(label)

  labels = [elem for i, elem in enumerate(list_labels) if i not in idx_to_remove]
  sequences = [elem for i, elem in enumerate(list_sequences) if i not in idx_to_remove]
  return labels, np.array(sequences)

print(test_labels)
_test_labels = [label.split() for label in test_labels]
test_labels2, test_pad_sequences = remove_sentences(_test_labels, test_pad_sequences_old)


test_numerical_labels = [label_encoder.transform(t) for t in test_labels2]
test_pad_labels = pad_sequences(test_numerical_labels, maxlen=max_sequence_length, padding="post")




_val_labels = [label.split() for label in val_labels]
val_labels2, val_pad_sequences = remove_sentences(_val_labels, val_pad_sequences_old)

val_numerical_labels = [label_encoder.transform(t) for t in val_labels2]
val_pad_labels = pad_sequences(val_numerical_labels, maxlen=max_sequence_length, padding="post")


num_classes = len(unique_entities)
train_labels_one_hot = [to_categorical(a, num_classes +1) for a in train_pad_labels]
val_labels_one_hot = [to_categorical(a, num_classes +1) for a in val_pad_labels]
test_labels_one_hot = [to_categorical(a, num_classes +1) for a in test_pad_labels]

[' O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name', ' O B-depart_date.month_name B-depart_date.day_number O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_time.time_relative B-depart_time.time I-depart_time.time', ' O B-depart_date.month_name B-depart_date.day_number O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name', ' O O O O O O B-round_trip I-round_trip O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.month_name B-depart_date.day_number', ' O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name I-toloc.city_name O B-depart_date.month_name B-depart_date.day_number O B-airline_name I-airline_name', ' O O O O O B-fromloc.city_name O B-toloc.city_name B-round_trip I-round_trip O B-depart_date.day_name B-depart_time.period_of_day O B-depart_date.day_name B-depart_time.period_of_day', ' B-depart_date.day_name B-depart_tim

In [23]:
len(test_labels_one_hot[1])

46

In [24]:
print((test_pad_sequences))

[[ 12  37  25 ...   0   0   0]
 [  5 266  41 ...   0   0   0]
 [  5 266  41 ...   0   0   0]
 ...
 [ 12  46  25 ...   0   0   0]
 [ 12  37  25 ...   0   0   0]
 [ 88   7  14 ...   0   0   0]]


<h1><a name="section-three"> 3. Disseny del model i entrenament </a></h1>

---
<h1><a name="ex-five"><center> ✏ Exercici 5 ✏</a></h1>

De forma similar com féreu a la primera part de la pràctica us demanem que dissenyeu l'arquitectura i entreneu el model. Podeu fer servir una arquitectura similar. Ara bé, per capturar les dependències seqüencials podeu canviar la capa de GlobalMaxPooling1D per una LSTM.

In [25]:
s = train_pad_labels.shape
train_flat_labels = train_pad_labels.reshape(s[0]*s[1])
train_unq_labels = np.unique(train_flat_labels)
print(train_flat_labels.shape, train_unq_labels)

(187588,) [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]


In [26]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        return token_embeddings + position_embeddings





In [27]:
print("train_pad_sequences shape:", train_pad_sequences.shape)
print("train_labels_one_hot shape:", np.shape(train_labels_one_hot))

print("val_pad_sequences shape:", val_pad_sequences.shape)
print("val_labels_one_hot shape:", np.shape(val_labels_one_hot))



train_pad_sequences shape: (4078, 46)
train_labels_one_hot shape: (4078, 46, 120)
val_pad_sequences shape: (897, 46)
val_labels_one_hot shape: (897, 46, 120)


In [28]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_pad_labels.flatten()),
    y=train_pad_labels.flatten()
)

class_weights_dict = dict(enumerate(class_weights))
# A KERAS NO LI AGRADA SI LI PASSEM UNS WEIGHTS, PERQUÈ ESTEM FENT PREDICCIÓ TOKEN A TOKEN
# HEM D'UTILTIZAR SAMPLE WEIGHTS
# SINCERAMENT, NO SÉ SI ESTÀ BÉ, I HE PROVAT I TAMPOC MILLORA GAIRE

class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

sample_weights = np.ones(train_pad_labels.shape)
for class_label, weight in class_weights_dict.items():
    sample_weights[train_pad_labels == class_label] = weight

In [ ]:
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D, Dropout, Conv1D, GlobalAveragePooling1D, LayerNormalization, TimeDistributed, MaxPooling1D
from keras.metrics import F1Score

In [208]:
len(train_pad_sequences[0])

46

In [30]:
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D, Dropout, Conv1D, GlobalAveragePooling1D,  SimpleRNN, LSTM, GRU, Bidirectional, TimeDistributed

In [32]:
embedding_dim = 1000
vocab_size = num_words + 1
maxlen = max_sequence_length
embed_dim = embedding_dim  # Embedding dimension for token and positional embeddings
transformer_block = TransformerBlock(embed_dim=embed_dim, num_heads=2, ff_dim=embed_dim*4)
transformer_block2 = TransformerBlock(embed_dim=embed_dim, num_heads=2, ff_dim=embed_dim*4)

inputs = keras.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
x = transformer_block(x)
x = transformer_block2(x)
x = TimeDistributed(Dense(num_classes + 1, activation="softmax"))(x)  # Output layer

model = keras.Model(inputs=inputs, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 3
model.fit(train_pad_sequences, np.array(train_labels_one_hot), batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences, np.array(val_labels_one_hot)))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_pad_sequences, np.array(test_labels_one_hot), batch_size=batch_size)
print(f"Test accuracy: {accuracy}")

Epoch 1/3
128/128 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.7215 - loss: 4.0313 - val_accuracy: 0.7552 - val_loss: 3.9453
Epoch 2/3
128/128 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.7519 - loss: 3.9984 - val_accuracy: 0.7552 - val_loss: 3.9453
Epoch 3/3
 46/128 ━━━━━━━━━━━━━━━━━━━━ 2:05 2s/step - accuracy: 0.7524 - loss: 3.9914

KeyboardInterrupt: 

In [50]:
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D, Dropout, Conv1D, GlobalAveragePooling1D, LayerNormalization, TimeDistributed, MaxPooling1D, Permute, Reshape, MaxPooling2D, Conv2D
from keras.metrics import F1Score

In [51]:
#TODO
embedding_dim = 30
vocab_size = num_words + 1

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))  # Embedding layer
model.add(Conv2D(64, 4, padding="same"))  # Convolutional layer

# Reshape to 4D: (batch_size, time_steps, filters, 1)
model.add(Reshape((-1, 64, 1)))

# Apply MaxPooling2D along the filter dimension only
model.add(MaxPooling2D(pool_size=(1, 2)))  # Reduces filter dimension by half

# Reshape back to 3D: (batch_size, time_steps, reduced_filters)
model.add(Reshape((-1, 32)))  # After pooling, the filter dimension is reduced by half

model.add(TimeDistributed(Dense(num_classes+1, activation="softmax")))
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
model.fit(train_pad_sequences, np.array(train_labels_one_hot), batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences, np.array(val_labels_one_hot)))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_pad_sequences, np.array(test_labels_one_hot), batch_size=batch_size)
print(f"Test accuracy: {accuracy}")


Epoch 1/10


ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 46, 30)

In [ ]:
#TODO
embedding_dim = 1000
vocab_size = num_words + 1

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) #layer 1
#model.add(Conv1D(64, 4, padding="same")) 
model.add(LSTM(64, return_sequences=True)) #layer 2
model.add(TimeDistributed(Dense(num_classes+1, activation="softmax")))
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
# SAMPLE WEIGHT ENLLOC DE CLASS WEIGHT
model.fit(train_pad_sequences, np.array(train_labels_one_hot), batch_size=batch_size,  sample_weight = sample_weights, epochs=epochs,  validation_data=(val_pad_sequences, np.array(val_labels_one_hot)))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_pad_sequences, np.array(test_labels_one_hot), batch_size=batch_size)
print(f"Test accuracy: {accuracy}")


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 14s 65ms/step - accuracy: 0.7412 - loss: 5.1594 - val_accuracy: 0.8000 - val_loss: 1.0401
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - accuracy: 0.8069 - loss: 2.8844 - val_accuracy: 0.8253 - val_loss: 0.8543
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.8396 - loss: 1.5329 - val_accuracy: 0.8728 - val_loss: 0.6706
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.8803 - loss: 1.3195 - val_accuracy: 0.8988 - val_loss: 0.5319
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - accuracy: 0.9035 - loss: 0.6562 - val_accuracy: 0.9094 - val_loss: 0.4517
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.9191 - loss: 0.4019 - val_accuracy: 0.9124 - val_loss: 0.4018
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.9210 - loss: 0.3513 - val_accuracy: 0.9231 - val_loss: 0.3234
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 10s 79ms/step - accuracy: 0.9327 - loss: 0.2573 - val

In [ ]:
# PER FER SERVIR F1_SCORE, el de Keras.metrics crec que no va, l'hem de calcular nosaltres a mà
from tensorflow.keras import backend as K

def f1(y_true, y_pred):
    # Fet amb el chat nidea de si esta be
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

#TODO
embedding_dim = 1000
vocab_size = num_words + 1

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) #layer 1
#model.add(Conv1D(64, 4, padding="same")) 
model.add(LSTM(64, return_sequences=True)) #layer 2
model.add(TimeDistributed(Dense(num_classes+1, activation="softmax")))
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])

# Train the model
batch_size = 32
epochs = 10
model.fit(train_pad_sequences, np.array(train_labels_one_hot), batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences, np.array(val_labels_one_hot)))


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - accuracy: 0.8633 - f1: 0.6525 - loss: 1.3905 - val_accuracy: 0.9443 - val_f1: 0.9633 - val_loss: 0.2760
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.9521 - f1: 0.9690 - loss: 0.2281 - val_accuracy: 0.9693 - val_f1: 0.9731 - val_loss: 0.1684
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.9779 - f1: 0.9796 - loss: 0.1261 - val_accuracy: 0.9788 - val_f1: 0.9811 - val_loss: 0.1209
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9860 - f1: 0.9862 - loss: 0.0809 - val_accuracy: 0.9827 - val_f1: 0.9842 - val_loss: 0.1013
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.9890 - f1: 0.9896 - loss: 0.0596 - val_accuracy: 0.9849 - val_f1: 0.9862 - val_loss: 0.0895
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.9916 - f1: 0.9920 - loss: 0.0449 - val_accuracy: 0.9862 - val_f1: 0.9869 - val_loss: 0.0828
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 43

ValueError: too many values to unpack (expected 2)

In [229]:

# Evaluate the model on the test set
loss, accuracy, f1_score = model.evaluate(test_pad_sequences, np.array(test_labels_one_hot), batch_size=batch_size)
print(f"Test accuracy: {accuracy}")
print(f"Test f1: {f1_score}")


28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9893 - f1: 0.9894 - loss: 0.0711
Test accuracy: 0.9894866943359375
Test f1: 0.9890897870063782


In [230]:
preds = model.predict(test_pad_sequences)

28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step


In [231]:
preds.shape

(885, 46, 120)

In [ ]:
preds = model.predict(test_pad_sequences)
def preds_to_index(preds, seq_lens):
  '''
  Turn predictions to numerical indexes, flatten the sentences and discard padding.
  '''
  idx_preds = []
  for pred, seq_len in zip(preds,seq_lens):
      for l in range(seq_len):
        idx_preds.append(np.argmax(pred[l]))
  return idx_preds

test_labels_idx = preds_to_index(test_labels_one_hot,[len(a) for a in test_sequences])
preds_idx = preds_to_index(preds, [len(a) for a in test_sequences])

print(classification_report(test_labels_idx, preds_idx, zero_division=1.0))

Com ja sabeu, NER es una tasca on les dades estan molt desbalancejades. La gran majoria de les nostres etiquetes serán 'O' (outside). Es pot donar el cas on el model tingui una accuracy molt alta predint sempre 'O'.

Per mesurar millor com de bó és el nostre model, calcularem la F1 score per cada classe, així com la mitjana (macro average). Quan prepareu el vostre document, heu de reportar aquesta mitjana com a mètrica del vostre model, no l'acuracy.

In [233]:
test_labels_idx = preds_to_index(test_labels_one_hot,[len(a) for a in test_sequences])
preds_idx = preds_to_index(preds, [len(a) for a in test_sequences])

print(classification_report(test_labels_idx, preds_idx, zero_division=1.0))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1549
           1       1.00      0.79      0.88        24
           2       1.00      0.66      0.79        29
           3       0.94      0.97      0.96        79
           4       1.00      0.12      0.22         8
           5       1.00      0.06      0.11        18
           6       1.00      0.00      0.00         1
           7       0.70      1.00      0.82         7
           8       1.00      0.67      0.80         3
           9       1.00      0.33      0.50         3
          13       1.00      1.00      1.00         2
          14       1.00      0.00      0.00         3
          15       0.75      0.92      0.83        13
          16       0.93      0.93      0.93        14
          17       0.64      0.63      0.64        43
          18       0.96      1.00      0.98        24
          19       1.00      1.00      1.00         6
          20       1.00    

A continuació podeu veure algunes prediccions del model.

In [234]:
probs = model.predict(test_pad_sequences)
_predicted_labels = np.argmax(probs, axis=2)

predicted_labels = [list(label_encoder.inverse_transform(label)) for label in _predicted_labels]

for i in range(0, 10):
    print('Sentence: ', test_sentences[i])
    print('Original label: ', test_labels[i])
    print('Predicted label: ', predicted_labels[i])
    print()

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Sentence:  i would like to find a flight from charlotte to las vegas that makes a stop in st. louis
Original label:   O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name
Predicted label:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'O', 'O', 'O', 'O', 'O', 'B-stoploc.city_name', 'I-stoploc.city_name', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']

Sentence:  on april first i need a ticket from tacoma to san jose departing before 7 am
Original label:   O B-depart_date.month_name B-depart_date.day_number O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_time.time_relative B-depart_time.time

---

 <h1><a name="ex-six"><center> ✏ Exercici 6 ✏ </a></h1>

Modifiqueu els següents paràmetres del model anterior i analitzeu com afecten a la seva *accuracy*:

 1. **Mida dels Embeddings.** Proveu diferents mides d'*Embeddings* i observeu com canvia l'*accuracy* del model. Heu d'explicar les vostres conclusions.

 2. **Xarxes Convolucionals.** Afegiu capes convolucionals al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació a l'hora d'escollir-los. Recordeu, que també podeu provar diferents configuracions de *pooling*.

 3. **Xarxes Recurrents.**  Afegiu capes recurrents al vostre model (LSTM, GRU). Expliqueu amb detall els valors que heu provat i la vostra motivació.

 4. ** Transformer.** Afegiu blocs de Transformer al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació.

 5. **Regularització.** Quan proveu configuracions amb més paràmetres veureu que el model comença a tenir *overfitting* molt prompte durant l'entrenament. Afegiu *Dropout* al vostre model. Heu d'explicar la vostra decisió de valors i de posició dins de la xarxa.


 6. **Balancejat de les classes.** Si analitzeu el dataset, veureu que la freqüència de les classes està molt desbalancejada. Keras us permet afegir un pes per a cada classe a l'hora de calcular la loss (Mireu el paràmetre "class_weigth" a la documentació https://keras.io/api/models/model_training_apis/). Calculeu un pes per a cada classe i afegiu-lo al mètode fit del vostre model.

 ---

---

<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Experiments i Resultats (amb raonament).
3. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.

 ---